# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets_1 import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [6]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'NS/LNA GDESEQMBALCTKRS(OGA)A E TL',
 'calculationPrice': 'tops',
 'open': None,
 'openTime': None,
 'openSource': 'flacioif',
 'close': None,
 'closeTime': None,
 'closeSource': 'lofciifa',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 134.86,
 'latestSource': 'IEX real time price',
 'latestTime': '11:28:54 AM',
 'latestUpdate': 1658136961725,
 'latestVolume': None,
 'iexRealtimePrice': 139.646,
 'iexRealtimeSize': 104,
 'iexLastUpdated': 1685959683600,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 136,
 'previousVolume': 69103960,
 'change': 0.415,
 'changePercent': 0.00307,
 'volume': None,
 'iexMarketPercent': 0.02028292479991977,
 'iexVolume': 52326

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [8]:
price = data['latestPrice']
pe_ratio = data['peRatio']

37.7

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [9]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [24]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
            index = my_columns
            ),
            ignore_index = True
        )
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,137.23,54.05,N/A
1,AAL,21.52,-1.18,N/A
2,AAP,207.29,28.63,N/A
3,AAPL,135.72,36.69,N/A
4,ABBV,114.60,40.99,N/A
...,...,...,...,...
500,YUM,119.08,42.06,N/A
501,ZBH,178.91,-276.78,N/A
502,ZBRA,512.31,54.69,N/A
503,ZION,55.74,8.66,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [25]:
final_dataframe.sort_values('Price-to-Earnings Ratio',inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index',axis = 1, inplace=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,634.980,5.24,N/A
1,ALL,123.670,7.25,N/A
2,UNM,29.935,7.44,N/A
3,EBAY,63.930,7.85,N/A
4,AFL,54.540,8.32,N/A
5,ZION,55.740,8.66,N/A
6,COO,415.800,9.34,N/A
7,KIM,21.084,9.35,N/A
8,CE,158.180,9.73,N/A
9,CTL,11.000,9.89,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [27]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [29]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [32]:
position_size = float(portfolio_size)/len(final_dataframe.index)

for row in final_dataframe.index:
    final_dataframe.loc[row,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row,'Price'])

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,634.980,5.24,31
1,ALL,123.670,7.25,161
2,UNM,29.935,7.44,668
3,EBAY,63.930,7.85,312
4,AFL,54.540,8.32,366
5,ZION,55.740,8.66,358
6,COO,415.800,9.34,48
7,KIM,21.084,9.35,948
8,CE,158.180,9.73,126
9,CTL,11.000,9.89,1818


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [59]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
print(data)
#Price-to-earnings ratio
pe_Ratio = data[symbol]['quote']['peRatio']
#Price-to-book ratio
pb_Ratio = data[symbol]['advanced-stats']['priceToBook']
#Price-to-sales ratio
ps_Ratio = data[symbol]['advanced-stats']['priceToSales']
#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda
print(ev_to_ebitda)
#Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit
print(ev_to_gross_profit)

ConnectionError: HTTPSConnectionPool(host='sandbox.iexapis.com', port=443): Max retries exceeded with url: /stable/stock/market/batch?symbols=AAPL&types=quote,advanced-stats&token=Tpk_059b97af715d417d9f49f50b51b1c448 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000238F68F3248>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [64]:
rv_columns =[
            'Ticker',
            'Price',
            'Number of Shares to Buy',
            'Price-to-Earning Ratio',
            'PE Percentile',
            'Price-to-Book Ratio',
            'PB Percentile',
            'Price-to-Sale Ratio',
            'PS Percentile',
            'EV/EBITDA',
            'EV/EBITDA Percentile',
            'EV/GP',
            'EV/GP Percentile',
            'RV Score'
            ]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        #if(enterprise_value == None):
        #    enterprise_value = 0.1
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        #if(ebitda == None):
        #    ebitda = 0.1
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        #if(gross_profit == None):
        #    gross_profit = 0.1

        try:
            ev_to_ebitda = enterprise_value/ebitda
        except:
            ev_to_ebitda = np.NaN

        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except:
            ev_to_gross_profit = np.NaN

        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['advanced-stats']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A'
                ],
                index = rv_columns
            ),
            ignore_index = True
        )


In [65]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sale Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.670,N/A,50.9885,N/A,8.78,N/A,7.41,N/A,33.429803,N/A,14.065003,N/A,N/A
1,AAL,20.880,N/A,-1.51939,N/A,-2.05,N/A,0.7869,N/A,-4.351374,N/A,2.266648,N/A,N/A
2,AAP,202.415,N/A,27.0752,N/A,3.82,N/A,1.3,N/A,13.436322,N/A,2.910633,N/A,N/A
3,AAPL,138.920,N/A,35.5716,N/A,34.23,N/A,7.66,N/A,25.748842,N/A,19.648618,N/A,N/A
4,ABBV,115.100,N/A,43.5047,N/A,15.13,N/A,4.35,N/A,14.361026,N/A,9.017828,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,123.146,N/A,40.3641,N/A,-4.5,N/A,6.5,N/A,25.815898,N/A,10.949312,N/A,N/A
501,ZBH,180.450,N/A,-277.117,N/A,3.03,N/A,5.43,N/A,33.067139,N/A,8.785415,N/A,N/A
502,ZBRA,499.820,N/A,53.7132,N/A,12.86,N/A,6.01,N/A,35.541792,N/A,13.635346,N/A,N/A
503,ZION,53.360,N/A,8.49612,N/A,1.2,N/A,2.61,N/A,6.719674,N/A,2.802747,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [68]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]


,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sale Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,230.670,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
42,APA,18.250,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,280.800,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,33.806,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.340,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,67.356,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,39.010,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,179.850,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2335.390,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [73]:
for column in ['Price-to-Earning Ratio','Price-to-Book Ratio','Price-to-Sale Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(),inplace=True)


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [74]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sale Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [78]:
from scipy.stats import percentileofscore as score
metrics =   {
    'Price-to-Earning Ratio' : 'PE Percentile',
    'Price-to-Book Ratio' : 'PB Percentile',
    'Price-to-Sale Ratio' : 'PS Percentile',
    'EV/EBITDA' : 'EV/EBITDA Percentile',
    'EV/GP' : 'EV/GP Percentile',
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row,metrics[metric]] = score(rv_dataframe[metric],rv_dataframe.loc[row,metric])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sale Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.670,N/A,50.988525,82.9703,8.78,74.6535,7.4100,78.2178,33.429803,86.7327,14.065003,77.2277,N/A
1,AAL,20.880,N/A,-1.519389,15.0495,-2.05,4.55446,0.7869,5.74257,-4.351374,3.9604,2.266648,8.11881,N/A
2,AAP,202.415,N/A,27.075194,47.7228,3.82,45.7426,1.3000,14.6535,13.436322,30.495,2.910633,11.6832,N/A
3,AAPL,138.920,N/A,35.571620,63.5644,34.23,95.2475,7.6600,79.2079,25.748842,76.8317,19.648618,91.0891,N/A
4,ABBV,115.100,N/A,43.504740,76.8317,15.13,87.3267,4.3500,57.2277,14.361026,36.4356,9.017828,51.2871,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,123.146,N/A,40.364142,73.4653,-4.50,4.35644,6.5000,74.6535,25.815898,77.2277,10.949312,66.1386,N/A
501,ZBH,180.450,N/A,-277.117344,0.990099,3.03,37.6238,5.4300,67.9208,33.067139,86.5347,8.785415,50.297,N/A
502,ZBRA,499.820,N/A,53.713166,83.7624,12.86,84.5545,6.0100,72.2772,35.541792,88.1188,13.635346,75.6436,N/A
503,ZION,53.360,N/A,8.496118,16.8317,1.20,9.90099,2.6100,36.2376,6.719674,5.94059,2.802747,10.8911,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [80]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentile = []
    for metric in metrics.keys():
        value_percentile.append(rv_dataframe.loc[row,metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentile)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sale Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.670,N/A,50.988525,82.9703,8.78,74.6535,7.4100,78.2178,33.429803,86.7327,14.065003,77.2277,79.9604
1,AAL,20.880,N/A,-1.519389,15.0495,-2.05,4.55446,0.7869,5.74257,-4.351374,3.9604,2.266648,8.11881,7.48515
2,AAP,202.415,N/A,27.075194,47.7228,3.82,45.7426,1.3000,14.6535,13.436322,30.495,2.910633,11.6832,30.0594
3,AAPL,138.920,N/A,35.571620,63.5644,34.23,95.2475,7.6600,79.2079,25.748842,76.8317,19.648618,91.0891,81.1881
4,ABBV,115.100,N/A,43.504740,76.8317,15.13,87.3267,4.3500,57.2277,14.361026,36.4356,9.017828,51.2871,61.8218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,123.146,N/A,40.364142,73.4653,-4.50,4.35644,6.5000,74.6535,25.815898,77.2277,10.949312,66.1386,59.1683
501,ZBH,180.450,N/A,-277.117344,0.990099,3.03,37.6238,5.4300,67.9208,33.067139,86.5347,8.785415,50.297,48.6733
502,ZBRA,499.820,N/A,53.713166,83.7624,12.86,84.5545,6.0100,72.2772,35.541792,88.1188,13.635346,75.6436,80.8713
503,ZION,53.360,N/A,8.496118,16.8317,1.20,9.90099,2.6100,36.2376,6.719674,5.94059,2.802747,10.8911,15.9604


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [82]:
rv_dataframe.sort_values('RV Score',ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True,inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sale Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.590,N/A,7.710260,16.4356,0.5638,4.9505,0.4498,1.9802,3.701956,4.75248,0.446357,0.39604,5.70297
1,FTI,7.100,N/A,-1.010046,15.6436,0.8063,5.94059,0.2481,0.990099,2.104716,4.15842,1.286876,3.16832,5.9802
2,MCK,203.842,N/A,-7.622940,11.0891,-66.9700,0.39604,0.1361,0.594059,7.731590,7.52475,2.871748,11.4851,6.21782
3,AIG,48.900,N/A,-6.884506,11.8812,0.6425,5.34653,0.9560,8.31683,7.364068,6.73267,0.922893,1.9802,6.85149
4,MET,63.370,N/A,11.051138,18.8119,0.7789,5.54455,0.8218,6.33663,5.599455,5.14851,0.818552,1.18812,7.40594
5,AAL,20.880,N/A,-1.519389,15.0495,-2.0500,4.55446,0.7869,5.74257,-4.351374,3.9604,2.266648,8.11881,7.48515
6,F,12.270,N/A,-37.407136,5.14851,1.5000,15.5446,0.3720,1.58416,5.820460,5.34653,2.665950,10.297,7.58416
7,ALL,125.596,N/A,7.154906,16.0396,1.3700,12.4752,0.8602,6.73267,2.665693,4.55446,0.825433,1.38614,8.23762
8,ABC,123.700,N/A,-7.787253,10.8911,-35.8000,1.18812,0.1283,0.39604,9.237621,12.6733,4.287544,20.9901,9.22772
9,DXC,32.710,N/A,-2.926852,13.6634,1.4300,13.8614,0.4631,2.37624,6.101721,5.74257,2.773671,10.6931,9.26733


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [83]:
portfolio_input()

In [86]:
position_size = float(portfolio_size)/len(rv_dataframe.index)

for i in rv_dataframe.index:
    rv_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe.loc[i,'Price'])
rv_dataframe

C:\Users\ayush\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sale Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.590,1748,7.710260,16.4356,0.5638,4.9505,0.4498,1.9802,3.701956,4.75248,0.446357,0.39604,5.70297
1,FTI,7.100,7042,-1.010046,15.6436,0.8063,5.94059,0.2481,0.990099,2.104716,4.15842,1.286876,3.16832,5.9802
2,MCK,203.842,245,-7.622940,11.0891,-66.9700,0.39604,0.1361,0.594059,7.731590,7.52475,2.871748,11.4851,6.21782
3,AIG,48.900,1022,-6.884506,11.8812,0.6425,5.34653,0.9560,8.31683,7.364068,6.73267,0.922893,1.9802,6.85149
4,MET,63.370,789,11.051138,18.8119,0.7789,5.54455,0.8218,6.33663,5.599455,5.14851,0.818552,1.18812,7.40594
5,AAL,20.880,2394,-1.519389,15.0495,-2.0500,4.55446,0.7869,5.74257,-4.351374,3.9604,2.266648,8.11881,7.48515
6,F,12.270,4074,-37.407136,5.14851,1.5000,15.5446,0.3720,1.58416,5.820460,5.34653,2.665950,10.297,7.58416
7,ALL,125.596,398,7.154906,16.0396,1.3700,12.4752,0.8602,6.73267,2.665693,4.55446,0.825433,1.38614,8.23762
8,ABC,123.700,404,-7.787253,10.8911,-35.8000,1.18812,0.1283,0.39604,9.237621,12.6733,4.287544,20.9901,9.22772
9,DXC,32.710,1528,-2.926852,13.6634,1.4300,13.8614,0.4631,2.37624,6.101721,5.74257,2.773671,10.6931,9.26733


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [88]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer,sheet_name = 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [92]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [93]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-to-Earnings Ratio', float_template],
    'E': ['PE Percentile', percent_template],
    'F': ['Price-to-Book Ratio', float_template],
    'G': ['PB Percentile',percent_template],
    'H': ['Price-to-Sales Ratio', float_template],
    'I': ['PS Percentile', percent_template],
    'J': ['EV/EBITDA', float_template],
    'K': ['EV/EBITDA Percentile', percent_template],
    'L': ['EV/GP', float_template],
    'M': ['EV/GP Percentile', percent_template],
    'N': ['RV Score', percent_template]
}

for column in column_formats:
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}',25,column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [95]:
writer.save()